In [ ]:
import kagglehub

path = kagglehub.dataset_download("manith04/ddd-processed-1-training-frames-type-1")
path = kagglehub.dataset_download("manith04/ddd-processed-2-training-frames-type-1")
path = kagglehub.dataset_download("manith04/ddd-processed-validation-frames-type-1")


100%|██████████| 4.14G/4.14G [03:12<00:00, 23.1MB/s]

Extracting files...


100%|██████████| 3.37G/3.37G [02:38<00:00, 22.8MB/s]

Extracting files...


100%|██████████| 1.57G/1.57G [01:12<00:00, 23.2MB/s]

Extracting files...


In [ ]:
!ls  /root/.cache/kagglehub/datasets/manith04/ddd-processed-validation-frames-type-1/versions/1

004  022  026  030


In [ ]:
!uv pip install "Pillow<10"

Using Python 3.12.12 environment at: /usr
Audited 1 package in 85ms


In [ ]:
!ls /root/.cache/kagglehub/datasets/manith04/ddd-processed-1-training-frames-type-1/versions/1

001  002  005  006  008  009  012  013	015  020


In [ ]:
# !uv pip uninstall pillow torchvision
# !uv pip  install torch==2.2.0 torchvision==0.15.2 torchaudio==2.2.0 "Pillow<10"
# !uv  pip install "Pillow<10" --force-reinstall
!uv pip install torchvision
!uv pip install -U pillow

Using Python 3.12.12 environment at: /usr
Audited 1 package in 92ms
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 29ms
Prepared 1 package in 84ms
Uninstalled 1 package in 1ms
Installed 1 package in 4ms
 - pillow==9.5.0
 + pillow==12.1.1


In [ ]:
from PIL import Image, UnidentifiedImageError
import torch
from torch.utils.data import Dataset
import os

from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import os
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import v2 as T
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import ConcatDataset



class DrowsySequenceDataset(Dataset):
    def __init__(self, root_dir, seq_len=15, transform=None):
        """
        Args:
            root_dir (str): Root directory containing subject folders.
            seq_len (int): Number of consecutive frames per sequence.
            transform (callable, optional): Optional transform to apply to each frame.
        """
        self.samples = []
        self.seq_len = seq_len
        self.transform = transform

        # Iterate over subjects
        for subject in os.listdir(root_dir):
            subject_path = os.path.join(root_dir, subject)
            if not os.path.isdir(subject_path):
                continue

            # Iterate over scenarios
            for scenario in os.listdir(subject_path):
                scenario_path = os.path.join(subject_path, scenario)
                if not os.path.isdir(scenario_path):
                    continue

                # Iterate over videos (or frames directly under scenario)
                for video in os.listdir(scenario_path):
                    video_path = os.path.join(scenario_path, video)
                    if not os.path.isdir(video_path):
                        video_path = scenario_path  # Handle case frames are directly under scenario

                    frame_files = sorted([
                        f for f in os.listdir(video_path)
                        if f.lower().endswith((".jpg", ".jpeg", ".png"))
                    ])

                    # Skip videos shorter than seq_len
                    if len(frame_files) < seq_len:
                        continue

                    # Create sequences
                    for i in range(len(frame_files) - seq_len + 1):
                        self.samples.append((video_path, frame_files[i:i + seq_len]))

        print(f"[Dataset] Total sequences found: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_dir, frame_list = self.samples[idx]
        images = []

        for fname in frame_list:
            img_path = os.path.join(video_dir, fname)
            try:
                img = Image.open(img_path)  # Pillow ≤9 works fine
                if img.mode != "L":
                    img = img.convert("L")  # convert to grayscale

                if self.transform:
                    img = self.transform(img)

                # Ensure [C, H, W]
                if isinstance(img, torch.Tensor) and img.ndim == 2:
                    img = img.unsqueeze(0)

                images.append(img)

            except (UnidentifiedImageError, OSError) as e:
                print(f"[Warning] Could not load {img_path}: {e}")
                # Add a zero tensor placeholder
                images.append(torch.zeros(1, 224, 224))

        images = torch.stack(images)  # [seq_len, C, H, W]

        # Label from last frame filename, e.g., "001_glasses_nonsleepyCombination_000000_0.jpg"
        last_fname = frame_list[-1]
        label = int(last_fname.rsplit("_", 1)[-1].split(".")[0])

        return images, torch.tensor(label, dtype=torch.long)

In [ ]:
class CNNEncoder(nn.Module):
    def __init__(self, feature_dim=512, pretrained=True):
        super().__init__()
        mobilenet = models.mobilenet_v3_small(pretrained=True)
        old_weights = mobilenet.features[0][0].weight          # Change first conv to accept 1 channel (grayscale)

        mobilenet.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)

        with torch.no_grad():
            mobilenet.features[0][0].weight[:] = old_weights.mean(dim=1, keepdim=True)          # Initialize weights by averaging RGB weights

        self.features = mobilenet.features          # Remove classifier, keep convolution layers only
        self.pool = nn.AdaptiveAvgPool2d((1, 1))           # Global average pooling
        self.fc = nn.Linear(1280, feature_dim)          # Project 1280 → feature_dim

        for param in self.features.parameters():
          param.requires_grad = False

    def forward(self, x):
        # x: (B*T, 1, H, W)
        x = self.features(x)        # (B*T, 1280, H', W')
        x = self.pool(x)            # (B*T, 1280, 1, 1)
        x = x.flatten(1)            # (B*T, 1280)
        x = self.fc(x)              # (B*T, feature_dim)
        return x




class CausalTemporalConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1):
        super().__init__()

        self.padding = (kernel_size - 1) * dilation

        self.conv = nn.Conv1d(
            in_channels, out_channels, kernel_size, padding=self.padding, dilation=dilation
        )

        self.norm = nn.GroupNorm(
            num_groups=8, num_channels=out_channels
        )

        self.relu = nn.ReLU()

        self.residual = (
            nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()
        )

    def forward(self, x):
        # x: (B, C, T)
        out = self.conv(x)

        # Remove future padding (causal trimming)
        out = out[:, :, :-self.padding]

        out = self.norm(out)
        out = self.relu(out)

        return out + self.residual(x)





class CNNTemporalConv(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()

        # Frame encoder
        self.encoder = CNNEncoder(feature_dim)

        # Temporal network
        self.tcn = nn.Sequential(
            CausalTemporalConvBlock(feature_dim, 256, kernel_size=3, dilation=1),
            CausalTemporalConvBlock(256, 256, kernel_size=3, dilation=2),
            CausalTemporalConvBlock(256, 128, kernel_size=3, dilation=4),
        )

        # Final classifier (only last time step)
        self.classifier = nn.Linear(128, 1)

    def forward(self, x):
        """
        x: (B, T, 3, H, W)
        """

        B, T, C, H, W = x.shape

        # Merge batch and time
        x = x.view(B * T, C, H, W)

        # Extract frame features
        features = self.encoder(x)           # (B*T, F)

        # Restore time dimension
        features = features.view(B, T, -1)   # (B, T, F)

        # Conv1d expects (B, C, T)
        features = features.transpose(1, 2)  # (B, F, T)

        # Temporal convolution
        tcn_out = self.tcn(features)         # (B, 128, T)

        # Take LAST time step
        last_feature = tcn_out[:, :, -1]     # (B, 128)

        logits = self.classifier(last_feature)  # (B, 1)

        return logits.squeeze(1)  # (B,)



In [ ]:

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

transform = T.Compose([
    T.Resize((112, 112)),
    T.ToImage(),                        # PIL → Tensor (still uint8)
    T.ToDtype(torch.float32, scale=1/255),
    T.Normalize(mean=imagenet_mean, std=imagenet_std)
])


train_ds1 = DrowsySequenceDataset("/root/.cache/kagglehub/datasets/manith04/ddd-processed-1-training-frames-type-1/versions/1", transform=transform)
train_ds2 = DrowsySequenceDataset("/root/.cache/kagglehub/datasets/manith04/ddd-processed-2-training-frames-type-1/versions/1", transform=transform)
train_ds = ConcatDataset([train_ds1, train_ds2])
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)

val_ds = DrowsySequenceDataset("/root/.cache/kagglehub/datasets/manith04/ddd-processed-validation-frames-type-1/versions/1", transform=transform)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

#test_ds = DrowsySequenceDataset("/kaggle/input/", transform=transform)
#test_loader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

device = torch.device("cuda")


model = CNNTemporalConv().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

epochs = 51
best_val_loss = float("inf")


In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

    for frames, labels in progress_bar:
        frames = frames.to(device)
        labels = labels.float().to(device)

        outputs = model(frames)
        loss = criterion(outputs, labels.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Update progress bar with current loss
        progress_bar.set_postfix({"Loss": loss.item()})

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Avg Loss: {avg_loss:.4f}")

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for frames, labels in val_loader:
            frames = frames.to(device)
            labels = labels.float().to(device)

            outputs = model(frames)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = (torch.sigmoid(outputs) > 0.5).int()
            correct += (preds == labels.int()).sum().item()
            total += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    print(f"Epoch {epoch+1} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Save last model
    torch.save(model.state_dict(), "last_model.pth")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print(f"Epoch {epoch+1}: Best model saved!")
